In [ ]:
# Instale as bibliotecas com o requirements.txt ------------------------------------
import warnings
warnings.simplefilter(action = 'ignore')
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [ ]:
# Execute esse bloco apenas se estiver usando o Google Colab para execução --------------------------
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Carregando o dataset ----------------------------------
pasta = '/content/drive/My Drive/Mestrado/Public_Contracts/'

df = pd.read_parquet(pasta + 'Notas_Fiscais_Itens_2023.parquet')
df.head()

,natureza_operacao,data_emissao,cnpj_cpf_emitente,razao_social_emitente,uf_emitente,mun_emitente,nome_destinatario,descricao_produto,ncm_produto,quantidade,valor_unitario,valor_total,classe,text
0,Outra saida merc./prest.serv. nao especif.,2023-01-01,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,SC,ITAJAI,MINISTERIO DA JUSTICA E SEGURANCA PUBLICA,MLTD201LXAZ CARTUCHO DE TONER PRETO 20K PAGINAS,Cartuchos de revelador (toners),1.0,169.58,169.58,0,[CLS] Natureza da operacao: Outra saida merc./...
1,Remessa de bem p/conta contrato de comodato ou...,2023-01-01,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,SC,ITAJAI,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,HP GABINETE METALICO,Outras partes e acessórios para aparelhos de f...,1.0,560.88,560.88,0,[CLS] Natureza da operacao: Remessa de bem p/c...
2,VENDA DE MARCADORIA ADQUIRIDA DE TERCEIROS,2023-01-01,34849096000189,"SEEK COMERCIO DE LIVROS, JORNAIS E REVISTAS LTDA",RJ,RIO DE JANEIRO,UNIVERSIDADE FEDERAL DO DELTA DO PARNAIBA UFDPAR,METODOS QUANTITATIVOS APLICADOS A CONTABILIDADE,"Outros livros, brochuras e impressos semelhantes",12.0,82.60,991.20,0,[CLS] Natureza da operacao: VENDA DE MARCADORI...
3,VENDA DE MARCADORIA ADQUIRIDA DE TERCEIROS,2023-01-01,34849096000189,"SEEK COMERCIO DE LIVROS, JORNAIS E REVISTAS LTDA",RJ,RIO DE JANEIRO,UNIVERSIDADE FEDERAL DO DELTA DO PARNAIBA UFDPAR,GESTAO ESTRATEGICA DE ARMAZENAMENTO,"Outros livros, brochuras e impressos semelhantes",12.0,78.40,940.80,0,[CLS] Natureza da operacao: VENDA DE MARCADORI...
4,VENDA DE MARCADORIA ADQUIRIDA DE TERCEIROS,2023-01-01,34849096000189,"SEEK COMERCIO DE LIVROS, JORNAIS E REVISTAS LTDA",RJ,RIO DE JANEIRO,UNIVERSIDADE FEDERAL DO DELTA DO PARNAIBA UFDPAR,RELACOES INTERNACIONAIS DA ASIA E DA AFRICA,"Outros livros, brochuras e impressos semelhantes",12.0,88.20,1058.40,0,[CLS] Natureza da operacao: VENDA DE MARCADORI...


In [ ]:
# Removendo linhas com NA's ---------------------------------
df = df.dropna().reset_index(drop = True)
df = df[['text', 'classe']].rename(columns = {'classe':'label'})
len(df)

5980558

In [ ]:
print('Dados de treino:\n',df.groupby('label', as_index = False).size())

Dados de treino:
    label     size
0      0  5731120
1      1   249438


In [ ]:
# Criando dataset balanceado com N quantidades de cada classe selecionadas de forma random ----------------------
samples_per_class = 10000

# Verificar se há exemplos suficientes em cada classe
count_label_0 = df[df['label'] == 0].shape[0]
count_label_1 = df[df['label'] == 1].shape[0]

if count_label_0 >= samples_per_class and count_label_1 >= samples_per_class:

    df_label_0 = df[df['label'] == 0].sample(n=samples_per_class, random_state=42)

    df_label_1 = df[df['label'] == 1].sample(n=samples_per_class, random_state=42)

    df_amostra = pd.concat([df_label_0, df_label_1]).reset_index(drop=True)

else:
    
    print("Não há exemplos suficientes em uma das classes para realizar a amostragem desejada.")

In [ ]:
print('Dados de treino:\n',df_amostra.groupby('label', as_index = False).size())

Dados de treino:
    label   size
0      0  10000
1      1  10000


In [ ]:
# Verificar se o Colab está usando a GPU está disponível -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f'Usando o dispositivo: {device}')

In [ ]:
# Modelos testados -----------------------------------
device = 0 if torch.cuda.is_available() else -1
#classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)
#classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device=device)
#classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=device)
#classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", device=device)
#classifier = pipeline("zero-shot-classification", model="1231czx/llama3_it_ultra_list_and_bold500", device=device)
#classifier = pipeline("zero-shot-classification", model="openai-community/roberta-large-openai-detector", device=device)
classifier = pipeline("zero-shot-classification", model="openai-community/gpt2", device=device)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [ ]:
# Labels candidatas
classe_labels = ['positivo', 'negativo']

# Função de classificação zero-shot -----------------------------------
def zero_shot_classify(text):

    result = classifier(text, classe_labels, multi_label=False, device = device)

    # Obter a label prevista
    predicted_label = result['labels'][0]
    predicted_value = 1 if predicted_label == 'positivo' else 0

    # Obter a pontuação associada à classe 'positivo'
    scores = result['scores']  # Lista de scores correspondentes às labels
    label_scores = dict(zip(result['labels'], scores))
    positive_score = label_scores.get('positivo', 0)

    return predicted_value, positive_score

In [ ]:
results = df_amostra['text'].progress_apply(zero_shot_classify)

100%|██████████| 20000/20000 [05:48<00:00, 57.35it/s]


In [ ]:
# Separar as predições e as probabilidades em colunas diferentes -----------------------------
df_amostra['predicted_value'] = results.apply(lambda x: x[0]) 
df_amostra['positive_score'] = results.apply(lambda x: x[1])

In [ ]:
# Criar a matriz de confusão -----------------------------------
y_true = df_amostra['label'].values
y_pred = df_amostra['predicted_value'].values
y_scores = df_amostra['positive_score'].values

cm = confusion_matrix(y_true, y_pred, labels=[0,1])
print("Matriz de Confusão:")
print(cm)


# [[TN, FP],
# [FN, TP]]

Matriz de Confusão:
[[   7 9993]
 [  11 9989]]


In [ ]:
# Criar a matriz de custo -----------------------------------
cost_FP = 1  # Custo de Falso Positivo
cost_FN = 5  # Custo de Falso Negativo
cost_TP = 0  # Custo de Verdadeiro Positivo
cost_TN = 0  # Custo de Verdadeiro Negativo

tn, fp, fn, tp = cm.ravel()

# Calcular o custo total ------------------------------------
total_cost = (fp * cost_FP) + (fn * cost_FN) + (tp * cost_TP) + (tn * cost_TN)
print(f'Custo Total: {total_cost}')

Custo Total: 10048


In [ ]:
# Criar a matriz de custo -----------------------------------
cost_FP = 5  # Custo de Falso Positivo
cost_FN = 1  # Custo de Falso Negativo
cost_TP = 0  # Custo de Verdadeiro Positivo
cost_TN = 0  # Custo de Verdadeiro Negativo

tn, fp, fn, tp = cm.ravel()

# Calcular o custo total ------------------------------------
total_cost = (fp * cost_FP) + (fn * cost_FN) + (tp * cost_TP) + (tn * cost_TN)
print(f'Custo Total: {total_cost}')

Custo Total: 49976


In [ ]:
# Calcular as métricas -----------------------------------
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_scores)

print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {auc:.4f}")

Acurácia: 0.4998
Precisão: 0.4999
Recall: 0.9989
F1 Score: 0.6663
AUC: 0.4947
